In [20]:
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.2/7.2 MB 57.5 MB/s eta 0:00:00
     --------------------------------------- 37.0/37.0 MB 47.0 MB/s eta 0:00:00
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
  Running setup.py install for sklearn: started
  Running setup.py install for sklearn: finished with status 'done'


In [68]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

## Lendo os dados e separando as colunas que utilizaremos para a analise

In [69]:
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
reviews = reviews[["review_score", "review_comment_message"]]
reviews = reviews.dropna()
reviews.head()

,review_score,review_comment_message
3,5,Recebi bem antes do prazo estipulado.
4,5,Parabéns lojas lannister adorei comprar pela I...
9,4,aparelho eficiente. no site a marca do aparelh...
12,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,5,"Vendedor confiável, produto ok e entrega antes..."


Pegando apenas os reviews negativos, pois são neles aonde conseguimos descobrir o que a empresam pode melhorar para melhor atender os seus clientes 

In [70]:
# get the reviews with less than 3 stars
unfavorable = reviews[reviews.review_score <= 3]
unfavorable.reset_index(drop=True, inplace=True)
unfavorable.columns = ["score", "comment"]
unfavorable.head()


,score,comment
0,2,"GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI E..."
1,1,Péssimo
2,1,Não gostei ! Comprei gato por lebre
3,1,Sempre compro pela Internet e a entrega ocorre...
4,1,Nada de chegar o meu pedido.


aa

In [72]:
nltk.download('stopwords')

sw = stopwords.words('portuguese')
vec = TfidfVectorizer(max_features=500, stop_words=sw, min_df=3, ngram_range=(3, 3))
X = vec.fit_transform(list(unfavorable.comment))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arfel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
keywords = []
n_keywords = 5
for doc in range(X.shape[0]): # para cada documento
    # print("Indexando documento ", doc)
    wordlist = []
    for word in vec.vocabulary_.keys(): # para cada palavra
        word_idx = vec.vocabulary_[word]
        if X[doc, word_idx] > 0:
            wordlist.append( (X[doc,word_idx], word) )
    wordlist = sorted(wordlist, reverse=True)
    keywords.append(wordlist[0:n_keywords])

In [78]:
k = []
for i in keywords:
    for j in i:
        k.append(j)
k = set(k)
k = sorted(k, reverse=True)
print(k[0:20])

[(1.0, 'último dia prazo'), (1.0, 'vou devolver produto'), (1.0, 'vez recebi produto'), (1.0, 'veio outro produto'), (1.0, 'veio nota fiscal'), (1.0, 'veio mal embalado'), (1.0, 'veio incompleto faltando'), (1.0, 'veio faltando produto'), (1.0, 'veio faltando peças'), (1.0, 'veio faltando peça'), (1.0, 'veio faltando item'), (1.0, 'veio errado pedi'), (1.0, 'veio errado comprei'), (1.0, 'veio defeito funciona'), (1.0, 'veio cor errada'), (1.0, 'veio bem embalado'), (1.0, 'vai ser entregue'), (1.0, 'vai fazer mês'), (1.0, 'unidades recebi apenas'), (1.0, 'unidades produto recebi')]


Como podemos ver dos top 20 produtos 